# Rendering with MultibodyPlant Tutorial

For instructions on how to run these tutorial notebooks, please see the [README](./README.md) in this folder.

This shows examples of:
* Adding two separate IIWAs to a diagram
* Adding LCM visualization for Drake Visualizer
* Adding a camera with a VTK renderer
* Rendering color and label images (at zero configuration)
* Remapping labels

## Necessary Imports

In [ ]:
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

In [ ]:
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import RoleAssign, PerceptionProperties, ConnectDrakeVisualizer
from pydrake.geometry.render import (
    DepthCameraProperties,
    RenderLabel,
    MakeRenderEngineVtk,
    RenderEngineVtkParams,
)
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import world_model_instance, BodyIndex
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.sensors import RgbdSensor

## Define helper methods

In [ ]:
def xyz_rpy_deg(xyz, rpy_deg):
    """Shorthand for defining a pose."""
    rpy_deg = np.asarray(rpy_deg)
    return RigidTransform(RollPitchYaw(rpy_deg * np.pi / 180), xyz)

In [ ]:
reserved_labels = [
    RenderLabel.kDoNotRender, RenderLabel.kDontCare, RenderLabel.kEmpty, RenderLabel.kUnspecified]

def colorize_labels(image):
    """Colorizes labels."""
    # TODO(eric.cousineau): Revive and use Kuni's palette.
    cc = mpl.colors.ColorConverter()
    color_cycle = plt.rcParams["axes.prop_cycle"]
    colors = np.array([cc.to_rgb(c["color"]) for c in color_cycle])
    bg_color =[ 0, 0, 0]
    image = np.squeeze(image)
    background = np.zeros(image.shape[:2], dtype=bool)
    for label in reserved_labels:
        background |= image == int(label)
    color_image = colors[image % len(colors)]
    color_image[background] = bg_color
    return color_image

## Create diagram builder with plant and scene graph.

In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder)

## Add first IIWA at origin.

In [ ]:
iiwa_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf")
iiwa_1 = Parser(plant).AddModelFromFile(iiwa_file, model_name="iiwa_1")
plant.WeldFrames(
    plant.world_frame(), plant.GetFrameByName("iiwa_link_0", iiwa_1),
    X_AB=xyz_rpy_deg([0, 0, 0], [0, 0, 0]))

## Add second IIWA at next to first.

In [ ]:
iiwa_2 = Parser(plant).AddModelFromFile(iiwa_file, model_name="iiwa_2")
plant.WeldFrames(
    plant.world_frame(), plant.GetFrameByName("iiwa_link_0", iiwa_2),
    X_AB=xyz_rpy_deg([0, 1, 0], [0, 0, 0]))

## Add renderer.

In [ ]:
scene_graph.AddRenderer("renderer", MakeRenderEngineVtk(RenderEngineVtkParams()))

## Add camera with same color and depth properties.

In [ ]:
depth_prop = DepthCameraProperties(
    width=640, height=480, fov_y=np.pi/4,
    renderer_name="renderer",
    z_near=0.01, z_far=10.)

world_id = plant.GetBodyFrameIdOrThrow(plant.world_body().index())
X_WB = xyz_rpy_deg([4, 0, 0], [-90, 0, 90])
sensor = RgbdSensor(world_id, X_PB=X_WB, properties=depth_prop)
builder.AddSystem(sensor)
builder.Connect(
    scene_graph.get_query_output_port(),
    sensor.query_object_input_port())

## Add Drake Visualizer.

In [ ]:
ConnectDrakeVisualizer(builder, scene_graph)

## Finalize plant and build diagram.

In [ ]:
plant.Finalize()
diagram = builder.Build()

## Create context and get subsystem contexts.

In [ ]:
diagram_context = diagram.CreateDefaultContext()
sensor_context = sensor.GetMyMutableContextFromRoot(diagram_context)
sg_context = scene_graph.GetMyMutableContextFromRoot(diagram_context)

## Publish visualization message with a default context.

In [ ]:
Simulator(diagram).Initialize()

## Render color and label images using matplotlib
Note that this uses the default labeling scheme, using `body.index()`.

In [ ]:
color = sensor.color_image_output_port().Eval(sensor_context).data
label = sensor.label_image_output_port().Eval(sensor_context).data
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(color)
ax[1].imshow(colorize_labels(label))

## Change labels to model instance by simple remapping.

In [ ]:
label_by_model = label.copy()
for i in range(plant.num_bodies()):
    body = plant.get_body(BodyIndex(i))
    label_by_model[label == i] = int(body.model_instance())

plt.imshow(colorize_labels(label_by_model))

## *Not yet supported*: Changing labels via property reassignment.

The following code block is disabled because it will fail
with the following error:

```
AssignRole() for updating properties currently only supports proximity properties
```

The code that may eventually work (in some form):

```
query_object = scene_graph.get_query_output_port().Eval(sg_context)
inspector = query_object.inspector()
for g in inspector.GetAllGeometryIds():
    body = plant.GetBodyFromFrameId(inspector.GetFrameId(g))
    props = PerceptionProperties()
    props.AddProperty("label", "id", RenderLabel(int(body.model_instance()))) 
    scene_graph.AssignRole(
        sg_context, plant.get_source_id(), g, props, RoleAssign.kReplace)
label = sensor.label_image_output_port().Eval(sensor_context).data

plt.imshow(colorize_labels(label))
```